In [6]:
import sys
sys.path.append('yolo11_try/hengck')
from dataset import *

overlay_dir = "../raw/train/overlay/ExperimentRuns"
static_dir = "../raw/train/static/ExperimentRuns"
_id = "TS_5_4"
label = read_one_truth(_id, overlay_dir)
volume = read_one_data(_id, static_dir)

In [8]:
# https://www.kaggle.com/competitions/czii-cryo-et-object-identification/discussion/547350#3056534
# 粒子が重なり合わないように半径を調整する必要がありそう。オリジナルの1/10に調整
config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [0, 255, 0, 0],
            "radius": 10,
            "map_threshold": 0.0418
        },
        {
            "name": "beta-amylase",
            "is_particle": true,
            "pdb_id": "1FA2",
            "label": 2,
            "color":[0, 0, 255, 255],
            "radius": 10,
            "map_threshold": 0.035
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [0, 255, 0, 255],
            "radius": 10,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [0, 0, 255, 0],
            "radius": 10,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [0, 255, 255, 0],
            "radius": 10,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [0, 0, 0, 255],
            "radius": 10,
            "map_threshold": 0.201
        }
    ],

    "overlay_root": "/kaggle/working/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "/kaggle/input/czii-cryo-et-object-identification/train/static"
}"""

copick_config_path = "../proc/create_3d_mask/copick.config"
output_overlay = "../proc/create_3d_mask/overlay"

with open(copick_config_path, "w") as f:
    f.write(config_blob)
    

In [9]:
import os
import shutil

# Define source and destination directories
source_dir = '../raw/train/overlay/ExperimentRuns'
destination_dir = '../proc/create_3d_mask/overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_apo-ferritin.json
Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-amylase.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_beta-amylase.json
Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-galactosidase.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_beta-galactosidase.json
Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/ribosome.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_ribosome.json
Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/thyroglobulin.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_thyroglobulin.json
Copied ../raw/train/overlay/ExperimentRuns/TS_5_4/Picks/virus-like-particle.json to ../proc/create_3d_mask/overlay/TS_5_4/Picks/curation_0_virus-like-particle.json
Copied ../raw/train/overlay/ExperimentRuns/TS_69_2/Picks/apo-ferritin.json to ..

In [10]:
import copick
from deepfindET.entry_points import step1
from deepfindET.utils import copick_tools
config = '/kaggle/working/copick.config'

# Query Tomogram
voxel_size = 10 
tomogram_algorithm = 'denoised'

# Output Name for the Segmentation Targets
out_name = 'remotetargets'
out_user_id = 'deepfindET'
out_session_id = '0'

# Read Copick Directory
copickRoot = copick.from_file(config)

ModuleNotFoundError: No module named 'deepfindET'

In [10]:
[(obj.name, None, None, (obj.radius / voxel_size)) for obj in copickRoot.pickable_objects if obj.is_particle]

[('apo-ferritin', None, None, 1.0),
 ('beta-amylase', None, None, 1.0),
 ('beta-galactosidase', None, None, 1.0),
 ('ribosome', None, None, 1.0),
 ('thyroglobulin', None, None, 1.0),
 ('virus-like-particle', None, None, 1.0)]

In [11]:
# Query Train Protein Coordiantes and any Associated Segmentations
train_targets = {}

# Define protein targets with their respective radii
# We can Provide two forms of inputs, either 
# ('protein-name',radius) or ('protein-name', 'user-id', 'session-id', 'radius')
targets = [(obj.name, None, None, (obj.radius / voxel_size)) for obj in copickRoot.pickable_objects if obj.is_particle]

# Set run_ids to None, indicating that targets will be generated for the entire CoPick project by default.
# If specific Run-IDs were provided, this variable would contain a list of those IDs.
run_ids = None

In [12]:
for t in targets:
    obj_name, user_id, session_id, radius = t
    info = {
        "label": copickRoot.get_object(obj_name).label,
        "user_id": user_id,
        "session_id": session_id,
        "radius": radius,
        "is_particle_target": True,
    }
    train_targets[obj_name] = info


# Define segmentation target (e.g., membrane)
#seg_targets = [('membrane', None, None)]

# Generate segmentation target information
#for s in seg_targets:
#    obj_name, user_id, session_id = s
#    info = {
#        "label": copickRoot.get_object(obj_name).label,
#        "user_id": user_id,
#        "session_id": session_id,
#        "radius": None,       
#        "is_particle_target": False,                 
#    }
#    train_targets[obj_name] = info

# Call the create_train_targets function from step1 to generate the training targets for the 3D U-Net model.
# The function will use the parameters defined in the previous cells and the following inputs:
step1.create_train_targets(
    config,              # The configuration file path specifying various settings and parameters for the project.
    train_targets,       # A dictionary containing the target information for each protein or object to be segmented.
    run_ids,             # The list of Run-IDs for which to generate targets. None means targets for the entire project.
    voxel_size,          # The voxel size to be used in the tomogram data.
    tomogram_algorithm,  # The reconstruction algorithm used for the tomograms, e.g., 'wbp' (weighted back projection).
    out_name,            # The output name for the generated segmentation targets.
    out_user_id,         # The user ID under which the output targets will be saved.
    out_session_id,      # The session ID associated with the output, typically used for tracking purposes.
)

Processing Run: TS_5_4
Annotating 140 objects ...

Processing Run: TS_69_2
Annotating 143 objects ...

Processing Run: TS_6_4
Annotating 193 objects ...

Processing Run: TS_6_6
Annotating 143 objects ...

Processing Run: TS_73_6
Annotating 217 objects ...

Processing Run: TS_86_3
Annotating 225 objects ...

Processing Run: TS_99_9
Annotating 208 objects ...



In [13]:
train_targets

{'apo-ferritin': {'label': 1,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True},
 'beta-amylase': {'label': 2,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True},
 'beta-galactosidase': {'label': 3,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True},
 'ribosome': {'label': 4,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True},
 'thyroglobulin': {'label': 5,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True},
 'virus-like-particle': {'label': 6,
  'user_id': None,
  'session_id': None,
  'radius': 1.0,
  'is_particle_target': True}}

In [14]:
# Option 1: Query All RunIDs
# Retrieve all available Run-IDs from the CoPick project. This generates a list of Run-IDs by iterating over all runs in copickRoot.
run_ids = [run.name for run in copickRoot.runs]

# Option 2: Manually Specify Specific Run
# Define a specific Run-ID manually. This is useful for extracting volumes for a specific run.
runID = 'TS_6_4'

# Retrieve the specific run object from CoPick using the manually specified Run-ID.
copick_run = copickRoot.get_run(runID)

# Extract the segmentation target associated with the specified run.
# The function get_copick_segmentation retrieves the segmentation data (e.g., target volume) based on the run object,
# segmentation name, user ID, and session ID.
train_target = copick_tools.get_copick_segmentation(
    copick_run,                 # The run object obtained from CoPick for the specific Run-ID.
    segmentationName='remotetargets',  # The name of the segmentation target to retrieve.
    userID='deepfindET',        # The user ID under which the segmentation data is saved.
    sessionID='0'               # The session ID associated with the segmentation data.
)

# Retrieve the tomogram associated with the specified Run-ID from the CoPick project.
# The function get_copick_tomogram extracts the tomogram data, using the voxel size, algorithm, and Run-ID.
train_tomogram = copick_tools.get_copick_tomogram(
    copickRoot,                 # The root object for the CoPick project, containing all runs and associated data.
    voxelSize=voxel_size,       # The voxel size to be used for retrieving the tomogram.
    tomoAlgorithm='wbp',        # The reconstruction algorithm used for the tomogram, e.g., 'wbp' (weighted back projection).
    tomoID=runID                # The specific Run-ID for which the tomogram is being retrieved.
)

In [15]:
run_ids = [run.name for run in copickRoot.runs]


for runID in run_ids:
    copick_run = copickRoot.get_run(runID)


    train_target = copick_tools.get_copick_segmentation(
        copick_run,                 # The run object obtained from CoPick for the specific Run-ID.
        segmentationName='remotetargets',  # The name of the segmentation target to retrieve.
        userID='deepfindET',        # The user ID under which the segmentation data is saved.
        sessionID='0'               # The session ID associated with the segmentation data.
    )
    volume = np.stack(train_target)
    D, H, W = volume.shape
    # pad to 640x640
    volume = np.pad(
        volume,
        [[0, 0], [0, 640 - H], [0, 640 - W]],
        mode="constant",
        constant_values=0,
    )
    np.save(f"{runID}.npy", volume)

In [16]:
# for download
!find ./ -name "*.npy" | tar -cf labels.tar -T -